In [1]:
import warnings

warnings.filterwarnings("ignore")

import os
import re
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
with open("d20_input.txt", "r") as f:
    lines = f.read().split("\n")
# lines = [(i[0], int(i[1:])) for i in lines]

In [3]:
lines[0], lines[-1]

('Tile 1321:', '')

In [4]:
# convert to tile dict
row = 0
tile_size = 10
pattern = r"\d+"
tile_dict = {}
while row < len(lines):
    l = lines[row]
    if l[:4] == "Tile":
        tile_num = int(re.findall(pattern, l)[0])
        tile_dict[tile_num] = []
        row += 1
        for i in range(tile_size):
            tile_dict[tile_num].append(
                [1 if j == "#" else 0 for j in lines[row + i]]
            )  # "#" as 1, "." as 0
        # convert to array
        tile_dict[tile_num] = np.array(tile_dict[tile_num])
        row += tile_size
    else:
        row += 1

In [5]:
len(tile_dict)

144

In [6]:
# the border is binary, then convert to int
def array_to_int(input_a):
    return int("".join([str(i) for i in input_a]), 2)

# flip, get 2 int
def array_rev_to_int(input_a):
    return [array_to_int(input_a), array_to_int(input_a[::-1])]

In [7]:
# get all the possible border int for each tile
tile_border_dict = {}
for k, v in tile_dict.items():
    tile_border_dict[k] = [
        array_rev_to_int(v[0, :]),
        array_rev_to_int(v[-1, :]),
        array_rev_to_int(v[:, 0]),
        array_rev_to_int(v[:, -1]),
    ]

In [8]:
tile_border_dict

{1321: [[468, 174], [656, 37], [343, 938], [96, 24]],
 2393: [[947, 823], [194, 268], [932, 151], [980, 175]],
 1301: [[335, 970], [178, 308], [156, 228], [1012, 191]],
 2647: [[981, 687], [662, 421], [723, 813], [726, 429]],
 2707: [[896, 7], [877, 731], [883, 827], [31, 992]],
 2521: [[168, 84], [147, 804], [202, 332], [29, 736]],
 3041: [[584, 73], [917, 679], [961, 527], [239, 988]],
 3529: [[246, 444], [263, 898], [236, 220], [41, 592]],
 1361: [[762, 381], [494, 478], [720, 45], [260, 130]],
 1303: [[488, 94], [75, 840], [78, 456], [149, 676]],
 2903: [[499, 830], [981, 687], [191, 1012], [855, 939]],
 3299: [[82, 296], [246, 444], [52, 176], [154, 356]],
 2659: [[783, 963], [206, 460], [990, 495], [576, 9]],
 3037: [[250, 380], [1003, 863], [33, 528], [105, 600]],
 2161: [[529, 545], [814, 467], [551, 913], [874, 347]],
 2767: [[210, 300], [153, 612], [416, 22], [225, 540]],
 1277: [[388, 134], [637, 761], [253, 764], [91, 872]],
 3593: [[398, 454], [28, 224], [182, 436], [116, 

In [9]:
# get common boarder pair of tiles
common_border_pair = {}
for k, v in tile_border_dict.items():
    for i, j in v:
        for o_k, o_v in tile_border_dict.items():
            if o_k != k:
                if (i in np.array(o_v)) or (j in np.array(o_v)):
                    if k in common_border_pair:
                        common_border_pair[k].append(o_k)
                    else:
                        common_border_pair[k] = [o_k]

In [10]:
common_border_pair

{1321: [1069, 2141, 3301, 2069],
 2393: [1733, 3019, 2689],
 1301: [3209, 1951, 2903],
 2647: [2903, 1499, 3209, 1319],
 2707: [3851, 1499, 1373, 2221],
 2521: [1129, 2687, 3733, 2551],
 3041: [2309, 2897, 2003],
 3529: [3299, 3917, 2371, 1429],
 1361: [1867, 1259, 2441, 2909],
 1303: [1699, 2477, 2557, 1423],
 2903: [1697, 2647, 1301, 1663],
 3299: [1423, 3529, 1471, 1699],
 2659: [1013, 1297, 3947],
 3037: [1129, 1483, 2551, 1543],
 2161: [2657, 2689],
 2767: [3709, 2441, 1259, 2857],
 1277: [1993, 3301, 3767, 2141],
 3593: [1699, 2423, 2557, 2179],
 2617: [1213, 2221, 3449, 1583],
 1103: [2621, 1697, 3557, 2203],
 3019: [1483, 2393, 2551],
 2909: [1361, 2269, 2437, 2791],
 3733: [3121, 3779, 1583, 2521],
 3851: [2707, 1019, 3167, 1439],
 3137: [2309, 3251, 2237, 2003],
 1697: [1103, 2903, 1951, 1789],
 3301: [1321, 2371, 3917, 1277],
 2237: [2377, 3137, 1609, 1549],
 2897: [2029, 1693, 3041],
 1423: [1303, 2297, 2131, 3299],
 1297: [2377, 2659, 1609],
 3463: [1289, 1531, 1759, 3727]

In [11]:
# if only 2 common borders, then it's corner
res = 1
for k, v in common_border_pair.items():
    if len(v) == 2:
        res *= k
res

20913499394191

In [12]:
# define a Tile class, with 4 border codes, and rot90, flip functions
class Tile:
    def __init__(self, tile_num, tile_arr):
        self.tile_num = tile_num
        self.tile_arr = tile_arr
        self.left_code = 0
        self.right_code = 0
        self.top_code = 0
        self.bottom_code = 0
        self._update_border_code()

    def _update_border_code(self):
        self.left_code = array_to_int(self.tile_arr[:, 0])
        self.right_code = array_to_int(self.tile_arr[:, -1])
        self.top_code = array_to_int(self.tile_arr[0, :])
        self.bottom_code = array_to_int(self.tile_arr[-1, :])

    def rot90(self, k=1):  # k: rot times
        self.tile_arr = np.rot90(self.tile_arr, k=k)
        self._update_border_code()

    def flip(self, axis=0):
        self.tile_arr = np.flip(self.tile_arr, axis=axis)
        self._update_border_code()

In [13]:
# convert to class Tile
tile_dict = {k: Tile(k, v) for k, v in tile_dict.items()}

In [14]:
# separate them into corner_li,border_li, inner_li
corner_li = []
border_li = []
inner_li = []
for k, v in common_border_pair.items():
    if len(v) == 2:
        corner_li.append(k)
        border_li.append(k)
    elif len(v) == 3:
        border_li.append(k)
    elif len(v) == 4:
        inner_li.append(k)
    else:
        print(f"error {k,v}")

In [15]:
corner_li

[2161, 2753, 2927, 1201]

In [16]:
# Init positions
positions = np.zeros(
    (int(len(tile_dict) ** 0.5), int(len(tile_dict) ** 0.5)), dtype="int"
)

In [17]:
positions[0, 0] = corner_li[0]  # let corner_li[0] as positions[0,0]
border_li.remove(positions[0, 0])
print(positions[0, 0])

2161


In [18]:
# put common_border_pair[k][0] as positions[0,1]
positions[0, 1] = common_border_pair[positions[0, 0]][0]
border_li.remove(positions[0, 1])
print(positions[0, 1])

2657


In [19]:
# find first row positions
for i in range(2, positions.shape[1]):
    for j in common_border_pair[positions[0, i - 1]]:
        if j in border_li:
            positions[0, i] = j
            border_li.remove(j)
            break

In [20]:
remain_li = [*border_li, *inner_li]
len(remain_li)

132

In [21]:
# find remain positions
for row in range(1, positions.shape[0]):
    for col in range(positions.shape[1]):
        for j in common_border_pair[positions[row - 1, col]]:
            if j in remain_li:
                positions[row, col] = j
                remain_li.remove(j)
                break

In [22]:
positions

array([[2161, 2657, 1097, 1907, 3167, 1373, 3527, 3209, 1301, 1951, 2203,
        2753],
       [2689, 3853, 2347, 1019, 3851, 2707, 1499, 2647, 2903, 1697, 1103,
        2621],
       [2393, 1733, 2687, 3121, 1439, 2221, 2089, 1319, 1663, 1789, 3557,
        3491],
       [3019, 2551, 2521, 3733, 1583, 2617, 3449, 3361, 1531, 2099, 2797,
        2699],
       [1483, 3037, 1129, 3779, 1553, 1213, 1151, 1759, 3463, 3727, 2129,
        2719],
       [2953, 1543, 3719, 2609, 2477, 2131, 1607, 2843, 1289, 3259, 2857,
        3413],
       [1693, 3623, 3761, 2557, 1303, 1423, 2297, 1171, 3797, 2441, 2767,
        3709],
       [2897, 2029, 2423, 3593, 1699, 3299, 1471, 2351, 1867, 1361, 1259,
        3769],
       [3041, 2003, 1249, 2179, 1429, 3529, 2371, 3767, 2791, 2909, 2437,
        1861],
       [2309, 3137, 3251, 2543, 3347, 3917, 3301, 1277, 1993, 2269, 2039,
        1187],
       [1549, 2237, 1609, 3947, 3407, 1069, 1321, 2141, 3547, 2579, 1063,
        2239],
       [1201, 2377, 1

In [23]:
# put [0,0] on the right position by rot or/and flip
k = positions[0, 0]
while not (
    (tile_dict[k].right_code in np.array(tile_border_dict[positions[0, 1]]))
    and (tile_dict[k].bottom_code in np.array(tile_border_dict[positions[1, 0]]))
):
    tile_dict[k].rot90()

In [24]:
tile_dict[k].right_code, tile_dict[k].bottom_code

(467, 551)

In [25]:
def get_reverse_code(c):
    return int("{0:010b}".format(c)[::-1], 2)

In [26]:
# rot/flip first row tiles
for i in range(1, positions.shape[1]):
    current_tile = tile_dict[positions[0, i]]
    target_code = tile_dict[positions[0, i - 1]].right_code
    while current_tile.left_code not in [target_code, get_reverse_code(target_code)]:
        current_tile.rot90()
    while current_tile.left_code != target_code:
        current_tile.flip()

In [27]:
# rot/flip other rows tiles
for row in range(1, positions.shape[0]):
    for col in range(positions.shape[1]):
        current_tile = tile_dict[positions[row, col]]
        target_code = tile_dict[positions[row - 1, col]].bottom_code
        while current_tile.top_code not in [target_code, get_reverse_code(target_code)]:
            current_tile.rot90()
        while current_tile.top_code != target_code:
            current_tile.flip(1)

In [28]:
# get final image
image = np.concatenate(
    [
        np.concatenate(
            [
                tile_dict[positions[row, col]].tile_arr[1:-1, 1:-1]
                for col in range(positions.shape[1])
            ],
            axis=1,
        )
        for row in range(positions.shape[0])
    ],
    axis=0,
)

In [29]:
image.shape

(96, 96)

In [30]:
image

array([[0, 0, 1, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 1]])

In [31]:
# now find monsters
"                  # "
"#    ##    ##    ###"
" #  #  #  #  #  #   "

' #  #  #  #  #  #   '

In [32]:
monster = np.array(
    [
        [1 if i == "#" else 0 for i in "                  # "],
        [1 if i == "#" else 0 for i in "#    ##    ##    ###"],
        [1 if i == "#" else 0 for i in " #  #  #  #  #  #   "],
    ]
)

In [33]:
def count_monster(input_image, monster):
    count = 0
    for row in range(input_image.shape[0] - monster.shape[0] + 1):
        for col in range(input_image.shape[1] - monster.shape[1] + 1):
            check_image = input_image[
                row : row + monster.shape[0], col : col + monster.shape[1]
            ]
            if np.all(check_image[np.where(monster == 1)] == 1): # check monster positions, very fast !!
                count += 1
                print(f"Monster {count} at the position ({row}-{col})")
    return count

In [34]:
n_monster = count_monster(image, monster)
print(f"Total monster num: {n_monster}")

Monster 1 at the position (4-73)
Monster 2 at the position (5-11)
Monster 3 at the position (10-50)
Monster 4 at the position (23-3)
Monster 5 at the position (34-53)
Monster 6 at the position (35-31)
Monster 7 at the position (38-68)
Monster 8 at the position (47-66)
Monster 9 at the position (48-24)
Monster 10 at the position (56-54)
Monster 11 at the position (57-2)
Monster 12 at the position (70-67)
Monster 13 at the position (75-70)
Monster 14 at the position (86-53)
Monster 15 at the position (91-22)
Total monster num: 15


In [35]:
# if n_monster==0, then you have to rot/flip image !!

image.sum() - n_monster * monster.sum()

2209